# Prerequisite
You need to have a working local installation of Ansys

In [ ]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

### Create the design in Metal
Setup a design of a given dimension. Dimensions will be respected in the design rendering. <br>
Note that the design size extends from the origin into the first quadrant.

In [ ]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
hfss = design.renderers.hfss

gui = MetalGUI(design)

Create a single transmon with one readout resonator and move it to the center of the chip previously defined.

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options =  dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))

gui.rebuild()
gui.autoscale()

Open a new Ansys window, connect to it, and add a driven modal design called SingleTransmon to the currently active project.

In [ ]:
hfss.connect_ansys()

In [ ]:
hfss.add_drivenmodal_design("SingleTransmon")

### Render the qubit from Metal into the SingleTransmon design in Ansys. <br>
Here, pin Q1_a is disconnected from the ground plane and the central junction is converted into a lumped port with an impedance of 70 Ohms. <br>
As a reminder, arguments are given as <br><br>
First parameter: List of components to render (empty list if rendering whole Metal design) <br>
Second parameter: List of pins (qcomp, pin) with open endcaps <br>
Third parameter: List of pins (qcomp, pin, impedance) to render as lumped ports <br>
Fourth parameter: List of junctions (qcomp, elt, impedance) to render as lumped ports <br>
Fifth parameter: List of junctions (qcomp, elt) to omit altogether during rendering

In [ ]:
hfss.render_design([], [('Q1', 'readout')], [], [('Q1', 'rect_jj', '70')], [])

Add a new driven modal simulation setup called NewSetup to the SingleTransmon design.

In [ ]:
hfss.add_drivenmodal_setup(name="NewSetup", max_passes=30)

All customizable parameters and their default values in add_drivenmodal_setup(): <br><br>
freq_ghz=5 (simulation frequency) <br>
name="Setup" (setup name) <br>
max_delta_s=0.1 (absolute value of maximum difference in scattering parameter S) <br>
max_passes=10 (maximum number of passes) <br>
min_passes=1 (minimum number of passes) <br>
min_converged=1 (minimum number of converged passes) <br>
pct_refinement=30 (percent refinement) <br>
basis_order=1 (basis order)

Verify that the currently active project, design, and setup names are correct.

In [ ]:
print(f"""
project_name = {hfss.pinfo.project_name}
design_name  = {hfss.pinfo.design_name}
setup_name   = {hfss.pinfo.setup_name}
""")

Next, add a frequency sweep called NewSweep to the NewSetup setup.

In [ ]:
hfss.add_sweep(setup_name="NewSetup", name="NewSweep", stop_ghz=13)

All customizable parameters and their default values in add_sweep(): <br><br>
setup_name="Setup" (name of setup) <br>
start_ghz=2.0 (starting frequency) <br>
stop_ghz=8.0 (stopping frequency) <br>
count=101 (total number of frequencies) <br>
step_ghz=None (frequency step size) <br>
name="Sweep" (name of sweep) <br>
type="Fast" (type of sweep) <br>
save_fields=False (whether or not to save fields)

In [ ]:
hfss.analyze_sweep('NewSweep', 'NewSetup')

In [ ]:
hfss.plot_s_params(['S11'])

Finally, disconnect from Ansys.

In [ ]:
hfss.disconnect_ansys()